# Stoplocaties clusteren en visualiseren op kaart
We hebben als eerst de data pre-processed. Daarna hebben verschillende parameters getest voor de DBScan. De output hebben we gevisualiseerd en via deze visualisatie kunnen we met de hand beslissen wat interessante parameters zijn voor de DBScan. Wij hebben als doel om stoplocaties te clusteren dus kiezen daarom parameters die een lage u-score hebben. Gezien uit de visualisaties blijkt dat er geen eenduidige solution space is tussen verschillende schepen, selecteren we de juiste parameters met de hand. 

De output uit de DBScan visualiseren op een interactieve kaart. Nu kunnen we in één opslag zien waar de stoplocaties zich bevinden op zee. 

In [ ]:
# Laden van packages
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import preprocessing
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, curdoc, show
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid
from bokeh.models.glyphs import VBar
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
import folium
import seaborn as sns

In [ ]:
# Laden pre-processed csv
df = pd.read_csv("sample_AIS_preprocessing.csv")

#### Voer DBScan uit met parameters die stoplocaties detecteren
Aan de DBScan geven we die parameters mee waarvan we in het vorige script hebben gezien dat deze leidt tot het clusteren van alleen stoplocaties en dat de varende punten worden gezien als ruis. 

Daarnaast kunnen we variëren in welke features we meegeven (uitleg features staan in 2de script). Mogelijke features zijn: t_latitude (latitude), t_longitude (longitude), VgNr (volgnummer met tijdselement), ID (volgnummer zonder tijdselement), t_speed (snelheid), angle_deg_diff (koersstabiliteit). 

In [ ]:
# Features in array-format meegeven. 
coords_speed = np.asarray(df[['t_latitude', 't_longitude', "VgNr", "t_speed"]])
# Features normaliseren, zodat ze met elkaar vergelijkbaar worden
coords = preprocessing.scale(coords_speed)

# Functie die DBScan runt, evalueert en output toevoegd aan dataframe
def dbs(df, coords, eps, min_samples):
    # DBScan uitvoeren
    model = DBSCAN(eps=eps, min_samples=min_samples, algorithm="auto").fit(coords)
    cluster_labels = model.labels_  #clusterlabels
    num_clusters = len(set(cluster_labels)) #aantal clusters
    df["cluster"] = cluster_labels #clusterlabels naar kolom df
    print('Number of clusters: {}'.format(num_clusters))

    # Model evalueren op basis van navigatiestatus
    ## Toewijzen of punt ruis of cluster is basis van uitkomen DBScan
    stoplocation = []
    for stop in df["cluster"]:
        if stop == -1:
            stoplocation.append(0)
        else:
            stoplocation.append(1)
    
    ## Toewijzen of punt ruis of cluster is basis van navigatiestatus
    ### NAVIGATIESTATUS 1=aan anker, 5=aangemeerd, 6=aan de grond
    stop_nav = []
    for stop in df["t_navstatus"]:
        if stop == 1:
            stop_nav.append(1)
        elif stop == 5:
            stop_nav.append(1)
        elif stop == 6:
            stop_nav.append(1)
        else:
            stop_nav.append(0)
    
    # Toevoegen aan kolom
    df["stoplocation"] = stoplocation
    df["stop_nav"] = stop_nav
    y_true = df["stop_nav"]
    y_pred = df["stoplocation"]
    
    # Evalueren model
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(y_true, y_pred))
    print("Completeness: %0.3f" % metrics.completeness_score(y_true, y_pred))
    print("V-measure: %0.3f" % metrics.v_measure_score(y_true, y_pred))
    print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(y_true, y_pred))
    print("Adjusted Mutual Information: %0.3f" % metrics.adjusted_mutual_info_score(y_true, y_pred))
    print("precision score is: %0.3f" % precision_score(y_true, y_pred, average="weighted"))
    print("recall score is: %0.3f" % recall_score(y_true, y_pred, average="weighted"))
    print("aantal punten per navigatiestatus:")
    print(df.groupby("t_navstatus")["t_mmsi"].count())        
    print("aantal punten per cluster:")
    print(df.groupby("cluster")["t_mmsi"].count())
    return(df)

# Run functie DBS met juiste parameters
df = dbs(df=df, coords=coords, eps=0.3, min_samples=30)

##### Evalueren uitkomst DBScan met visualisaties

In [ ]:
# Plot navigatiestatus: snelheid vs. tijd  
sns.scatterplot(x="VgNr", y="t_speed", hue="t_navstatus", data=df, legend="full")

In [ ]:
# Plot clusters: snelheid vs. tijd
sns.scatterplot(x="VgNr", y="t_speed", hue="cluster", data=df, legend="full")

In [ ]:
# Plot cluster: snelheid vs. koersstabiliteit
sns.scatterplot(x="t_speed", y="angle_deg_diff", hue="cluster", data=df, legend="full")

In [ ]:
#  Plot cluster: Volgnummer vs. koersstabiliteit
sns.scatterplot(x="VgNr", y="angle_deg_diff", hue="cluster", data=df, legend="full")

##### Plot clusters op kaart
Nu gaan we de clusters plotten op een interactieve kaart. De package die we hiervoor gebruiken is folium en leaflet. 

In [ ]:
# Creëer kleurmap per cluster (straks voor visualisatie kaart)
colors = []
for stop in df["cluster"]:
    if stop == -1:
        colors.append("black")
    elif stop == 0:
        colors.append("red")
    elif stop == 1:
        colors.append("orange")
    elif stop == 2:
        colors.append("pink")
    elif stop == 3:
        colors.append("blue")
    elif stop == 4:
        colors.append("beige")
    elif stop == 5:
        colors.append("purple")
    elif stop == 6:
        colors.append("gray")
    elif stop == 7:
        colors.append("white")
    elif stop == 8:
        colors.append("darkgreen")
    elif stop == 9:
        colors.append("yellow")
    elif stop == 10:
        colors.append("lightblue")
    else:
        colors.append("green")
df["colors"] = colors

In [ ]:
# Creëer interactieve kaart

# Pak de middelste lon/lat punt
mid_location = df['t_latitude'].mean(), df['t_longitude'].mean()

# Lat/lon naar list
coords_map = df[["t_latitude", "t_longitude"]].values.tolist()

# Assign labels en kleuren aan clusters
labels = "ID:"+df["VgNr"].astype(str) +' '+df["t_name"]
colors = df["colors"].values.tolist()

# Creëer een folium map met de coordinaten
m = folium.Map(location=mid_location, zoom_start=8)
for point in range(len(coords_map)):
    popup = folium.Popup(labels[point], parse_html=True) #labels
    folium.Circle(coords_map[point],
                  color=colors[point],
                  radius=5,
                  popup=popup).add_to(m)

# Sla kaart op als HTML 
m.save('sample_color_cluster.html')